In [ ]:
%run funcs.py

# global variables

# used to create id strings later
base_url = 'https://www.basketball-reference.com'

season_gamecount = 1

# precovid_seasons = ['1314', '1415', '1516', '1617', '1718', '1819']
# precovid_url_years = ['2014', '2015', '2016', '2017', '2018', '2019']
# postcovid_seasons = ['1920', '2021', '2122', '2223']
# postcovid_url_years = ['2020', '2021', '2022', '2023']

# post_covid_season_dict = {'1920': {'month_len': 8, 'final_month_gamecount': 83},
#                           '2021': {'month_len': 6, 'final_month_gamecount': 140},
#                           '2122': {'month_len': 7, 'final_month_gamecount': 83},
#                           '2223': {'month_len': 7, 'final_month_gamecount': 72}
#                          }

# used to create sql database table columns

info_columns = ['game_id', 'season', 'date', 'type', 'away_team', 'away_score', 'home_team', 'home_score', 'result']
num_columns = ['MP', 'FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-',
                'FG%', '3P%', 'FT%', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']

basic_stats = ['MP', 'FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-']
# advanced_stats = ['FG%', '3P%', 'FT%', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']

rand = 5

In [ ]:
# connect to sql database
conn = sqlite3.connect('data/temp/tester-tester.sqlite')
driver = webdriver.Chrome(service=svc,options=options)
time.sleep(rand)

# season_id = '1617'
# url_year = '2017'

# precovid_seasons = ['1617', '1718', '1819']
# precovid_url_years = ['2017', '2018', '2019']
# postcovid_seasons = ['1920', '2021', '2122', '2223']
# postcovid_url_years = ['2020', '2021', '2022', '2023']

In [ ]:
# query = 'SELECT * from player_stats_quarterly'
# game_info = pd.read_sql(query, con=conn)
# # account for team name change following the 2013-2014 season
# # game_info['away_team'].replace('CHA', 'CHO', inplace=True)
# # game_info['home_team'].replace('CHA', 'CHO', inplace=True)
# game_info

In [ ]:
start_url = 'https://www.basketball-reference.com/leagues/NBA_2020_games-august.html'

driver.get(start_url)
time.sleep(rand)
src = driver.page_source
parser = BeautifulSoup(src, 'lxml')
table = parser.find('div', attrs = {'class': 'table_container is_setup'})

In [ ]:
play_in_str = table.findAll('td', string='Play-In Game')

In [ ]:
play_in_str

In [ ]:
play_in_str = []

In [ ]:
if not play_in_str:
    print('no')
elif play_in_str:
    print('yes')

In [ ]:
play_in_rows =[]
if play_in_str:
    for cell in play_in_str:
        parent = cell.find_parent()
        row = int(parent['data-row'])
        play_in_rows.append(row)
else:
    print('no')

In [ ]:
# reg_games: [:up to first playin]
# playin = [playin[0]:playin[-1]+1]
# playoff = [playin[-1]+1:]

In [ ]:
    
season_id = '1920'
url_year = '2020'
season_gamecount = 1
game_type = 'regular'
month_url = 'https://www.basketball-reference.com/leagues/NBA_2020_games-august.html'
    
        
driver.get(month_url)
time.sleep(rand)
month_src = driver.page_source
month_parser = BeautifulSoup(month_src, 'lxml')
month_table = month_parser.find('div', attrs = {'class': 'table_container is_setup'})

#check if there are play-in games this month
play_in_str = month_table.findAll('td', string='Play-In Game')

# if no play-in games:
if not play_in_str:

    game_partial_urls = month_table.findAll('td', attrs = {'class': 'center', 'data-stat': 'box_score_text'})
    game_urls = [base_url + url.a['href'] for url in game_partial_urls]

    for i in range(len(game_urls)):

        # driver.quit()
        # time.sleep(rand)
        # driver = webdriver.Chrome(service=svc,options=options)
        # time.sleep(rand)
        # driver.maximize_window()
        driver.get(game_urls[i])
        time.sleep(rand)
        src = driver.page_source
        parser = BeautifulSoup(src, 'lxml')

        # game_info database:

        id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
        game_info = create_game_info(url=game_urls[i],
                                     season_id=season_id,
                                     season_gamecount=season_gamecount,
                                     game_type=game_type)
        # will use game_id with create_boxscores()
        game_id = game_info[0]
        team_info = create_team_info(id_table)
        # will use team_ids with merge_boxscores()
        team_ids = [team_info[0], team_info[2]]

        info_df = create_info_df(game_info=game_info,
                                 team_info=team_info,
                                 info_columns=info_columns)
        # write game info to sql database
        info_df.to_sql('game_info', con=conn, if_exists='append', index=False)

        # team/player databases:

        # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
        stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})

        player_box_list = [None, None, None, None]
        team_box_list = [None, None, None, None]

        # create team and player boxscores
        for i in range(len(stat_tables)):
            # split player and team boxscores
            player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)

        # team_stats database:

        # combine boxscore and advanced boxscore for each team
        away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
        team_boxes = pd.concat([away_team_box, home_team_box])
        team_boxes.reset_index(drop=True, inplace=True)
        # prepare numeric data
        team_boxes = change_dtypes(team_boxes, num_columns)
        # write to sql database
        team_boxes.to_sql('team_stats_game', con=conn, if_exists='append', index=False)

        # player_stats database:

        # combine boxscore and advanced boxscore for each team
        away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
        player_boxes = pd.concat([away_player_box, home_player_box])
        player_boxes.reset_index(drop=True, inplace=True)
        # prepare numeric data
        player_boxes = change_dtypes(player_boxes, num_columns)
        # create team totals for PIE calculation
        totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
        # add PIE column to player boxscore
        player_boxes = create_PIE(player_boxes, totals)
        # write to sql database
        player_boxes.to_sql('player_stats_game', con=conn, if_exists='append', index=False)

        
        button_banner = driver.find_element(By.CLASS_NAME, 'filter.switcher')

        buttons = button_banner.find_elements(By.CLASS_NAME, 'sr_preset')

        del buttons[6]
        del buttons[3]
        del buttons[0]

        game_length = len(buttons)

        button_codes = ['q1', 'q2', 'q3', 'q4', 'ot1', 'ot2', 'ot3', 'ot4', 'ot5', 'ot6']
        game_button_codes = button_codes[:game_length]

        for team_id in team_ids:
            for period in game_button_codes:

                id_string = f'box-{team_id}-{period}-basic'
                period_table = parser.find('table', attrs = {'id': id_string})

                player_box, team_box = create_boxscores(period_table, game_id=game_id)

                # team_stats database:

                team_box.rename(columns={'Starters': 'team'}, inplace=True)
                team_box['team'] = team_id
                team_box.insert(loc=1, column='period', value=period)

                team_box.reset_index(drop=True, inplace=True)
                team_box = change_dtypes(team_box, basic_stats)

                team_box.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)

                # player_stats database:

                player_box.insert(loc=2, column='team', value=team_id)
                player_box.insert(loc=1, column='period', value=period)

                player_box.reset_index(drop=True, inplace=True)
                player_box = change_dtypes(player_box, basic_stats)

                player_box.to_sql('player_stats_quarterly', con=conn, if_exists='append', index=False)

        # increase gamecount to create next game_id
        season_gamecount += 1
        time.sleep(rand)

elif play_in_str:

    play_in_rows = []
    for cell in play_in_str:
        parent = cell.find_parent()
        row = int(parent['data-row'])
        play_in_rows.append(row)
    
    first_play_in = play_in_rows[0]
    last_play_in = play_in_rows[-1]
    first_playoff = last_play_in + 1
    
    parent_table = play_in_str[0].find_parent().find_parent()
    slip_headers = parent_table.findAll('tr', attrs = {'class': 'thead'})
    
    for header in slip_headers:
        header_row = int(header['data-row'])
        if header_row < first_play_in:
            first_play_in -= 1
        if header_row < last_play_in:
            first_playoff -= 1
    
    game_partial_urls = month_table.findAll('td', attrs = {'class': 'center', 'data-stat': 'box_score_text'})
    reg_game_urls = [base_url + url.a['href'] for url in game_partial_urls[:first_play_in]]
    play_in_game_urls = [base_url + url.a['href'] for url in game_partial_urls[first_play_in:first_playoff]]
    playoff_game_urls = [base_url + url.a['href'] for url in game_partial_urls[first_playoff:]]


    for i in range(len(reg_game_urls)):

        # driver.quit()
        # time.sleep(rand)
        # driver = webdriver.Chrome(service=svc,options=options)
        # time.sleep(rand)
        # driver.maximize_window()
        driver.get(reg_game_urls[i])
        time.sleep(rand)
        src = driver.page_source
        parser = BeautifulSoup(src, 'lxml')

        # game_info database:

        id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
        game_info = create_game_info(url=reg_game_urls[i],
                                     season_id=season_id,
                                     season_gamecount=season_gamecount,
                                     game_type=game_type)
        # will use game_id with create_boxscores()
        game_id = game_info[0]
        team_info = create_team_info(id_table)
        # will use team_ids with merge_boxscores()
        team_ids = [team_info[0], team_info[2]]

        info_df = create_info_df(game_info=game_info,
                                 team_info=team_info,
                                 info_columns=info_columns)
        # write game info to sql database
        info_df.to_sql('game_info', con=conn, if_exists='append', index=False)

        # team/player databases:

        # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
        stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})

        player_box_list = [None, None, None, None]
        team_box_list = [None, None, None, None]

        # create team and player boxscores
        for i in range(len(stat_tables)):
            # split player and team boxscores
            player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)

        # team_stats database:

        # combine boxscore and advanced boxscore for each team
        away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
        team_boxes = pd.concat([away_team_box, home_team_box])
        team_boxes.reset_index(drop=True, inplace=True)
        # prepare numeric data
        team_boxes = change_dtypes(team_boxes, num_columns)
        # write to sql database
        team_boxes.to_sql('team_stats_game', con=conn, if_exists='append', index=False)

        # player_stats database:

        # combine boxscore and advanced boxscore for each team
        away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
        player_boxes = pd.concat([away_player_box, home_player_box])
        player_boxes.reset_index(drop=True, inplace=True)
        # prepare numeric data
        player_boxes = change_dtypes(player_boxes, num_columns)
        # create team totals for PIE calculation
        totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
        # add PIE column to player boxscore
        player_boxes = create_PIE(player_boxes, totals)
        # write to sql database
        player_boxes.to_sql('player_stats_game', con=conn, if_exists='append', index=False)

        
        button_banner = driver.find_element(By.CLASS_NAME, 'filter.switcher')

        buttons = button_banner.find_elements(By.CLASS_NAME, 'sr_preset')

        del buttons[6]
        del buttons[3]
        del buttons[0]

        game_length = len(buttons)

        button_codes = ['q1', 'q2', 'q3', 'q4', 'ot1', 'ot2', 'ot3', 'ot4', 'ot5', 'ot6']
        game_button_codes = button_codes[:game_length]

        for team_id in team_ids:
            for period in game_button_codes:

                id_string = f'box-{team_id}-{period}-basic'
                period_table = parser.find('table', attrs = {'id': id_string})

                player_box, team_box = create_boxscores(period_table, game_id=game_id)

                # team_stats database:

                team_box.rename(columns={'Starters': 'team'}, inplace=True)
                team_box['team'] = team_id
                team_box.insert(loc=1, column='period', value=period)

                team_box.reset_index(drop=True, inplace=True)
                team_box = change_dtypes(team_box, basic_stats)

                team_box.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)

                # player_stats database:

                player_box.insert(loc=2, column='team', value=team_id)
                player_box.insert(loc=1, column='period', value=period)

                player_box.reset_index(drop=True, inplace=True)
                player_box = change_dtypes(player_box, basic_stats)

                player_box.to_sql('player_stats_quarterly', con=conn, if_exists='append', index=False)

        # increase gamecount to create next game_id
        season_gamecount += 1

    # game_type = 'play-in'
    # for i in range(len(play_in_game_urls)):

    #     # driver.quit()
    #     # time.sleep(rand)
    #     # driver = webdriver.Chrome(service=svc,options=options)
    #     # time.sleep(rand)
    #     # driver.maximize_window()
    #     driver.get(play_in_game_urls[i])
    #     time.sleep(rand)
    #     src = driver.page_source
    #     parser = BeautifulSoup(src, 'lxml')

    #     # game_info database:

    #     id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
    #     game_info = create_game_info(url=play_in_game_urls[i],
    #                                  season_id=season_id,
    #                                  season_gamecount=season_gamecount,
    #                                  game_type=game_type)
    #     # will use game_id with create_boxscores()
    #     game_id = game_info[0]
    #     team_info = create_team_info(id_table)
    #     # will use team_ids with merge_boxscores()
    #     team_ids = [team_info[0], team_info[2]]

    #     info_df = create_info_df(game_info=game_info,
    #                              team_info=team_info,
    #                              info_columns=info_columns)
    #     # write game info to sql database
    #     info_df.to_sql('game_info', con=conn, if_exists='append', index=False)

    #     # team/player databases:

    #     # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
    #     stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})

    #     player_box_list = [None, None, None, None]
    #     team_box_list = [None, None, None, None]

    #     # create team and player boxscores
    #     for i in range(len(stat_tables)):
    #         # split player and team boxscores
    #         player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)

    #     # team_stats database:

    #     # combine boxscore and advanced boxscore for each team
    #     away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
    #     team_boxes = pd.concat([away_team_box, home_team_box])
    #     team_boxes.reset_index(drop=True, inplace=True)
    #     # prepare numeric data
    #     team_boxes = change_dtypes(team_boxes, num_columns)
    #     # write to sql database
    #     team_boxes.to_sql('team_stats_game', con=conn, if_exists='append', index=False)

    #     # player_stats database:

    #     # combine boxscore and advanced boxscore for each team
    #     away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
    #     player_boxes = pd.concat([away_player_box, home_player_box])
    #     player_boxes.reset_index(drop=True, inplace=True)
    #     # prepare numeric data
    #     player_boxes = change_dtypes(player_boxes, num_columns)
    #     # create team totals for PIE calculation
    #     totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
    #     # add PIE column to player boxscore
    #     player_boxes = create_PIE(player_boxes, totals)
    #     # write to sql database
    #     player_boxes.to_sql('player_stats_game', con=conn, if_exists='append', index=False)

        
    #     button_banner = driver.find_element(By.CLASS_NAME, 'filter.switcher')

    #     buttons = button_banner.find_elements(By.CLASS_NAME, 'sr_preset')

    #     del buttons[6]
    #     del buttons[3]
    #     del buttons[0]

    #     game_length = len(buttons)

    #     button_codes = ['q1', 'q2', 'q3', 'q4', 'ot1', 'ot2', 'ot3', 'ot4', 'ot5', 'ot6']
    #     game_button_codes = button_codes[:game_length]

    #     for team_id in team_ids:
    #         for period in game_button_codes:

    #             id_string = f'box-{team_id}-{period}-basic'
    #             period_table = parser.find('table', attrs = {'id': id_string})

    #             player_box, team_box = create_boxscores(period_table, game_id=game_id)

    #             # team_stats database:

    #             team_box.rename(columns={'Starters': 'team'}, inplace=True)
    #             team_box['team'] = team_id
    #             team_box.insert(loc=1, column='period', value=period)

    #             team_box.reset_index(drop=True, inplace=True)
    #             team_box = change_dtypes(team_box, basic_stats)

    #             team_box.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)

    #             # player_stats database:

    #             player_box.insert(loc=2, column='team', value=team_id)
    #             player_box.insert(loc=1, column='period', value=period)

    #             player_box.reset_index(drop=True, inplace=True)
    #             player_box = change_dtypes(player_box, basic_stats)

    #             player_box.to_sql('player_stats_quarterly', con=conn, if_exists='append', index=False)

    #     # increase gamecount to create next game_id
    #     season_gamecount += 1

    
    game_type = 'playoff'
    for i in range(len(playoff_game_urls)):

        # driver.quit()
        # time.sleep(rand)
        # driver = webdriver.Chrome(service=svc,options=options)
        # time.sleep(rand)
        driver.get(playoff_game_urls[i])
        time.sleep(rand)
        src = driver.page_source
        parser = BeautifulSoup(src, 'lxml')

        # game_info database:

        id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
        game_info = create_game_info(url=playoff_game_urls[i],
                                     season_id=season_id,
                                     season_gamecount=season_gamecount,
                                     game_type=game_type)
        # will use game_id with create_boxscores()
        game_id = game_info[0]
        team_info = create_team_info(id_table)
        # will use team_ids with merge_boxscores()
        team_ids = [team_info[0], team_info[2]]

        info_df = create_info_df(game_info=game_info,
                                 team_info=team_info,
                                 info_columns=info_columns)
        # write game info to sql database
        info_df.to_sql('game_info', con=conn, if_exists='append', index=False)

        # team/player databases:

        # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
        stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})

        player_box_list = [None, None, None, None]
        team_box_list = [None, None, None, None]

        # create team and player boxscores
        for i in range(len(stat_tables)):
            # split player and team boxscores
            player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)

        # team_stats database:

        # combine boxscore and advanced boxscore for each team
        away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
        team_boxes = pd.concat([away_team_box, home_team_box])
        team_boxes.reset_index(drop=True, inplace=True)
        # prepare numeric data
        team_boxes = change_dtypes(team_boxes, num_columns)
        # write to sql database
        team_boxes.to_sql('team_stats_game', con=conn, if_exists='append', index=False)

        # player_stats database:

        # combine boxscore and advanced boxscore for each team
        away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
        player_boxes = pd.concat([away_player_box, home_player_box])
        player_boxes.reset_index(drop=True, inplace=True)
        # prepare numeric data
        player_boxes = change_dtypes(player_boxes, num_columns)
        # create team totals for PIE calculation
        totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
        # add PIE column to player boxscore
        player_boxes = create_PIE(player_boxes, totals)
        # write to sql database
        player_boxes.to_sql('player_stats_game', con=conn, if_exists='append', index=False)

        # quarterly boxscores:
        button_banner = driver.find_element(By.CLASS_NAME, 'filter.switcher')

        # button order: Game, Q1, Q2, H1, Q3, Q4, H2, (OT1, OT2, ...)
        buttons = button_banner.find_elements(By.CLASS_NAME, 'sr_preset')

        del buttons[6]
        del buttons[3]
        del buttons[0]

        game_length = len(buttons)

        button_codes = ['q1', 'q2', 'q3', 'q4', 'ot1', 'ot2', 'ot3', 'ot4', 'ot5', 'ot6']
        game_button_codes = button_codes[:game_length]

        for team_id in team_ids:
            for period in game_button_codes:

                id_string = f'box-{team_id}-{period}-basic'
                period_table = parser.find('table', attrs = {'id': id_string})

                player_box, team_box = create_boxscores(period_table, game_id=game_id)

                # team_stats database:

                team_box.rename(columns={'Starters': 'team'}, inplace=True)
                team_box['team'] = team_id
                team_box.insert(loc=1, column='period', value=period)

                team_box = change_dtypes(team_box, basic_stats)

                team_box.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)

                # player_stats database:

                player_box.insert(loc=2, column='team', value=team_id)
                player_box.insert(loc=1, column='period', value=period)

                player_box = change_dtypes(player_box, basic_stats)

                player_box.to_sql('player_stats_quarterly', con=conn, if_exists='append', index=False)

        # increase gamecount to create next game_id
        season_gamecount += 1

In [ ]:
game_partial_urls[:first_play_in]

In [ ]:
len(game_partial_urls)

In [ ]:
#list comprehension:
# find parent (tbody), get the tr class thead elements (and their position: data row) for each 
# list comp: for each integer in play_in_rows, if it is between a certain value (based on thead data row values),
# subtract by the number of previous thead elements

In [ ]:
play_in_str[0]

In [ ]:
table_parent = play_in_str[0].find_parent().find_parent()

In [ ]:
table_parent

In [ ]:
slip_headers = table_parent.findAll('tr', attrs = {'class': 'thead'})
slip_header_count = len(slip_headers)
slip_headers[0]['data-row']

In [ ]:
slip_headers[0]

In [ ]:
for child in table_parent.children:
    print('l')

In [ ]:
ll = [1,2,3,4,6,7,9,10]
new_ll = [x if x<5 else x-1 if 5<x<8 else x-2 for x in ll]
new_ll

In [ ]:
play_in_rows = []
for cell in play_in_str:
    parent = cell.find_parent()
    row = int(parent['data-row'])
    play_in_rows.append(row)

first_play_in = play_in_rows[0]
last_play_in = play_in_rows[-1]
first_playoff = last_play_in + 1

parent_table = play_in_str[0].find_parent().find_parent()
slip_headers = parent_table.findAll('tr', attrs = {'class': 'thead'})

for header in slip_headers:
    header_row = int(header['data-row'])
    if header_row < first_play_in:
        first_play_in -= 1
    if header_row < last_play_in:
        first_playoff -= 1

game_partial_urls = month_table.findAll('td', attrs = {'class': 'center', 'data-stat': 'box_score_text'})
reg_game_urls = [base_url + url.a['href'] for url in game_partial_urls[:first_play_in]]
play_in_game_urls = [base_url + url.a['href'] for url in game_partial_urls[first_play_in:first_playoff]]
playoff_game_urls = [base_url + url.a['href'] for url in game_partial_urls[first_playoff:]]

In [ ]:
# game_partial_urls[0]
play_in_rows

In [ ]:
# play-in = 7,9,10
# headers = 5,8
pp = [1,2,3,4,6,7,9,10,11,12]
# 

In [ ]:
query = 'SELECT * from game_info'
game_info = pd.read_sql(query, con=conn)
# account for team name change following the 2013-2014 season
# game_info['away_team'].replace('CHA', 'CHO', inplace=True)
# game_info['home_team'].replace('CHA', 'CHO', inplace=True)
game_info[70:100]